# Pratice 2: Quantum Support Vector Machines

MQIST 2025/26: Quantum Computing and Machine Learning

Alfredo Chavert Sancho

Pedro Herrero Maldonado

## Data loading and preprocessing

The `load_breast_cancer` function from `sklearn.datasets` is used to load the breast cancer dataset. This dataset contains features computed from breast cancer biopsy images, along with labels indicating whether the cancer is malignant or benign. So this is a binary classification problem. There are 30 features in total and 569 samples which are divided into:

- 357 benign samples labeled as 1
- 212 malignant samples labeled as 0

As measure to evaluate the performance of our quantum neural network, we will use **recall** of the positive class labeled with 0, which is the ratio of correctly predicted positive malignant cases to the total actual positive instances in the dataset (true positives and false negatives). 
$$
\text{Recall} = \frac{\text{True Malignant}}{\text{True Malignant} + \text{False Benign}}
$$

This is particularly important in medical diagnosis tasks, where minimizing false negatives (i.e., failing to identify actual positive cases) is crucial. Therefore, our goal in this work is to maximize this metric in our experiments.

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data # features
y = data.target # labels

print("Original shape:", X.shape)

Original shape: (569, 30)


For the prepocessing the data to be used in the quantum neural network, we will use  ``StandardScaler`` from ``sklearn.preprocessing`` to normalize it  since all the data is numeric.

We will also use ``PCA`` from ``sklearn.decomposition`` to reduce the dimensionality of the data to reduce the number of qubits needed in the quantum neural network. This transformation helps in capturing the most significant variance in the data while reducing its complexity.

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Standardize (important for PCA)
# scaler = StandardScaler()
norm = StandardScaler() 
X_norm = norm.fit_transform(X)

# Apply PCA to reduce to 4 dimensions
pca = PCA(n_components = 4)
X_pca = pca.fit_transform(X_norm)

print("Shape after PCA:", X_pca.shape) # (150, 2)
print("Explained variance:", pca.explained_variance_ratio_)
print("Total variance explained:", pca.explained_variance_ratio_.sum())

Shape after PCA: (569, 4)
Explained variance: [0.44272026 0.18971182 0.09393163 0.06602135]
Total variance explained: 0.7923850582446098


The ``MinMaxScaler`` from ``sklearn.preprocessing`` is used to scale the data to a range between 0 and 1. This scaling is important because the quantum circuits used in the neural network often require input values to be within specific ranges for effective encoding into quantum states.

In [4]:
scaler = MinMaxScaler() 
X_scaled = scaler.fit_transform(X_pca)

Finally, we will split the dataset into training and testing sets using ``train_test_split`` from ``sklearn.model_selection``.

In [5]:
from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals

# Set the seed so results are reproducible
algorithm_globals.random_seed = 42

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=algorithm_globals.random_seed,)

## Classical SVM

Classical SVM is implemented using ``SVC`` from ``sklearn.svm`` with a radial basis function (RBF) kernel. The model is trained on the training set and evaluated on the test set. The recall metric is calculated to assess the performance of the classical SVM.

### Linear kernel

Fisrtly, we will implement a linear kernel SVM as a baseline model. The linear kernel is suitable for linearly separable data and serves as a reference point for comparing the performance of more complex kernels.

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

svc_linear = SVC(kernel = 'linear', gamma='auto')
svc_linear.fit(X_train, y_train)

y_svc_linear = svc_linear.predict(X_test)
report_svc_linear = classification_report(y_test, y_svc_linear)
print(report_svc_linear)

              precision    recall  f1-score   support

           0       1.00      0.88      0.94        43
           1       0.93      1.00      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.94      0.95       114
weighted avg       0.96      0.96      0.96       114



The recall obtained with the linear kernel SVM is 0.88

### Polynomial kernel

Next, we will implement a polynomial kernel SVM. The polynomial kernel allows for modeling non-linear decision boundaries by transforming the input space into a higher-dimensional space.

In [ ]:
svc_poly = SVC(kernel = 'poly', gamma='auto') 
svc_poly.fit(X_train, y_train)

y_svc_poly = svc_poly.predict(X_test)
report_svc_poly = classification_report(y_test, y_svc_poly)
print(report_svc_poly)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.62      1.00      0.77        71

    accuracy                           0.62       114
   macro avg       0.31      0.50      0.38       114
weighted avg       0.39      0.62      0.48       114



C:\Users\fredi\.conda\envs\qml\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\fredi\.conda\envs\qml\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\fredi\.conda\envs\qml\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


The recall obtained with the polynomial kernel SVM is 0.00 which is worse than the linear kernel. This suggests that the polynomial kernel may not be suitable for this particular dataset or that the chosen degree of the polynomial is not optimal.

### RBF kernel

Finally, we will implement an RBF kernel SVM. The RBF kernel is a popular choice for non-linear classification tasks as it can handle complex decision boundaries by mapping the input data into an infinite-dimensional space.

In [7]:
svc_rbf = SVC(kernel = 'rbf', gamma='auto')
svc_rbf.fit(X_train, y_train)

y_svc_rbf = svc_rbf.predict(X_test)
report_svc_rbf = classification_report(y_test, y_svc_rbf)
print(report_svc_rbf)

              precision    recall  f1-score   support

           0       1.00      0.88      0.94        43
           1       0.93      1.00      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.94      0.95       114
weighted avg       0.96      0.96      0.96       114



The recall obtained with the polynomial kernel SVM is 0.88 which is the same as the linear kernel.

The performance of the classical SVM with Linear, Polynomial, and RBF kernel is as follows:

| Kernel     | Recall (0) |
|------------|--------|
| Linear     | 0.88  |
| Polynomial | 0.00  |
| RBF        | 0.88  |

We can observe that both the  Linear and RBF kernels perform the best in terms of recall for positive cases marked as 0 for this dataset.



## Quantum SVM

In this section, we will implement a Quantum Support Vector Machine (QSVM) using the Qiskit library. QSVM leverages the principles of quantum computing to perform classification tasks, potentially offering advantages over classical SVMs, especially for complex datasets.

Let's try different quantum kernels/feature maps and compare their performance in terms of recall for the positive class labeled with 0.

### Z Feature Map

Let's start by implementing a QSVM using the Z Feature Map. The Z Feature Map encodes classical data into quantum states using rotations around the Z-axis of the Bloch sphere.

In [8]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZFeatureMap

num_features = X_pca.shape[1]

kernel = FidelityQuantumKernel(feature_map = ZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc_z = QSVC(quantum_kernel = kernel)
qsvc_z.fit(X_train, y_train)
y_pred_qsvc_z = qsvc_z.predict(X_test)

report_qsvc_z = classification_report(y_test, y_pred_qsvc_z)
print(report_qsvc_z)

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



We obtain a recall of 0.93 with this feature map, which is better than the classical SVM with linear and RBF kernels.

### ZZ Feature Map

The ZZ Feature Map extends the Z Feature Map by incorporating entangling gates between qubits, allowing for more complex data representations.

In [9]:
from qiskit.circuit.library import ZZFeatureMap

kernel = FidelityQuantumKernel(feature_map = ZZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc_zz = QSVC(quantum_kernel = kernel)
qsvc_zz.fit(X_train, y_train)
y_pred_qsvc_zz = qsvc_zz.predict(X_test)

report_qsvc_zz = classification_report(y_test, y_pred_qsvc_zz)
print(report_qsvc_zz)

              precision    recall  f1-score   support

           0       0.93      0.88      0.90        43
           1       0.93      0.96      0.94        71

    accuracy                           0.93       114
   macro avg       0.93      0.92      0.92       114
weighted avg       0.93      0.93      0.93       114



We get a recall of 0.88 with this feature map, which is the same as the classical SVM with linear and RBF kernels.

### Pauli Feature Map

Finally Pauli Feature Map uses a combination of Pauli gates to encode the data into quantum states, providing a richer representation of the input features.

In [10]:
from qiskit.circuit.library import PauliFeatureMap

kernel = FidelityQuantumKernel(feature_map = PauliFeatureMap(feature_dimension = num_features, reps = 2, paulis = ['Z', 'XX', 'ZXZ']))
qsvc_pauli = QSVC(quantum_kernel = kernel)
qsvc_pauli.fit(X_train, y_train)
y_pred_qsvc_pauli = qsvc_pauli.predict(X_test)

report_qsvc_pauli = classification_report(y_test, y_pred_qsvc_pauli)
print(report_qsvc_pauli)

              precision    recall  f1-score   support

           0       0.65      0.51      0.57        43
           1       0.74      0.83      0.78        71

    accuracy                           0.71       114
   macro avg       0.69      0.67      0.68       114
weighted avg       0.70      0.71      0.70       114



In this case, we achieve a recall of 0.51, which is lower than the previous quantum feature maps and the classical SVM with linear and RBF kernels.

### ZFeatureMap + C=0.1 + OVO

In [6]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZFeatureMap
from sklearn.metrics import classification_report
import time

num_features = X_train.shape[1]

kernel = FidelityQuantumKernel(feature_map = ZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc3 = QSVC(quantum_kernel = kernel, C = 0.1)
start = time.time()
qsvc3.fit(X_train, y_train)
elapsed3 = time.time() - start
y_pred_qsvc3 = qsvc3.predict(X_test)

report_qsvc3 = classification_report(y_test, y_pred_qsvc3)
print(report_qsvc3)
print(f"Training time: {round(elapsed3)} seconds")

              precision    recall  f1-score   support

           0       1.00      0.70      0.82        43
           1       0.85      1.00      0.92        71

    accuracy                           0.89       114
   macro avg       0.92      0.85      0.87       114
weighted avg       0.90      0.89      0.88       114

Training time: 250 seconds


### ZFeatureMap + C=10.0 + OVO

In [7]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZFeatureMap
from sklearn.metrics import classification_report
import time

num_features = X_train.shape[1]

kernel = FidelityQuantumKernel(feature_map = ZFeatureMap(feature_dimension = num_features, reps = 2))
qsvc4 = QSVC(quantum_kernel = kernel, C = 10.0)
start = time.time()
qsvc4.fit(X_train, y_train)
elapsed4 = time.time() - start
y_pred_qsvc4 = qsvc4.predict(X_test)

report_qsvc4 = classification_report(y_test, y_pred_qsvc4)
print(report_qsvc4)
print(f"Training time: {round(elapsed4)} seconds")

              precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

Training time: 261 seconds


## Conclusion

In this practice, we explored the implementation of Quantum Support Vector Machines (QSVM) using different quantum feature maps and compared their performance with classical SVMs on the breast cancer dataset. The results indicate that the choice of quantum feature map significantly impacts the model's performance. The Z Feature Map outperformed both the classical SVMs and other quantum feature maps, achieving the highest recall for the positive class labeled with 0. This suggests that certain quantum feature maps may be more effective in capturing the underlying patterns in the data.